# Kaggle Exploration Kickstarter

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

In [3]:
spark = SparkSession.builder.getOrCreate()

votre taille d'écran 131072x1 est fausse. Attendez-vous à des problèmes


22/11/27 21:02:26 WARN Utils: Your hostname, DESKTOP-NIT8A65 resolves to a loopback address: 127.0.1.1; using 172.25.180.10 instead (on interface eth0)
22/11/27 21:02:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/27 21:02:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.option("header", True).option("inferSchema", "True").csv("/home/mgard/Documents/pyspark/cours-spark-telecom/data/train_clean.csv")

In [5]:
print(f'Nombre de lignes: {df.count()}')
print(f"Nombre de colonnes: {len(df.columns)}")

Nombre de lignes: 108129
Nombre de colonnes: 14


In [6]:
df.show(n=10)

+--------------+--------------------+--------------------+------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|    project_id|                name|                desc|  goal|            keywords|disable_communication|country|currency|  deadline|state_changed_at|created_at|launched_at|backers_count|final_status|
+--------------+--------------------+--------------------+------+--------------------+---------------------+-------+--------+----------+----------------+----------+-----------+-------------+------------+
|kkst1451568084| drawing for dollars|I like drawing pi...|  20.0| drawing-for-dollars|                False|     US|     USD|1241333999|      1241334017|1240600507| 1240602723|            3|           1|
|kkst1474482071|Sponsor Dereck Bl...|I  Dereck Blackbu...| 300.0|sponsor-dereck-bl...|                False|     US|     USD|1242429000|      1242432018|1240960224| 1240975592|        

In [7]:
df.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- disable_communication: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: string (nullable = true)
 |-- state_changed_at: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- launched_at: string (nullable = true)
 |-- backers_count: integer (nullable = true)
 |-- final_status: integer (nullable = true)



### Chargement des données

L’ensemble des ressources nécessaires pour les prochaines questions se trouvent dans la documentation de Spark.

Chargez le fichier train_clean.csv dans un DataFrame. La première ligne du fichier donne le nom de chaque colonne (aka le header), on veut que cette ligne soit utilisée pour nommer les colonnes du dataFrame.

In [8]:
dfcasted = df.withColumn("goal", df.goal.cast("int")) \
             .withColumn("deadline", df.deadline.cast("int")) \
             .withColumn("state_changed_at", df.state_changed_at.cast("int")) \
             .withColumn("created_at", df.created_at.cast("int")) \
             .withColumn("launched_at", df.launched_at.cast("int")) \
             .withColumn("backers_count", df.backers_count.cast("int")) \
             .withColumn("final_status", df.final_status.cast("int"))

In [9]:
dfcasted.printSchema()

root
 |-- project_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- goal: integer (nullable = true)
 |-- keywords: string (nullable = true)
 |-- disable_communication: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- deadline: integer (nullable = true)
 |-- state_changed_at: integer (nullable = true)
 |-- created_at: integer (nullable = true)
 |-- launched_at: integer (nullable = true)
 |-- backers_count: integer (nullable = true)
 |-- final_status: integer (nullable = true)



## Cleaning

Certaines opérations sur les colonnes sont déjà implémentées dans Spark, mais il est souvent nécessaire de faire appel à des fonctions plus complexes. Dans ce cas on peut créer des UDFs (User Defined Functions) qui permettent d’implémenter de nouvelles opérations sur les colonnes. Voir la partie User Defined Functions du fichier spark_notes.md pour comprendre comment ça fonctionne.

Affichez une description statistique des colonnes de type Int :

In [10]:
dfcasted.select("goal", "backers_count", "final_status") \
        .describe() \
        .show()

+-------+-----------------+-------------------+--------------------+
|summary|             goal|      backers_count|        final_status|
+-------+-----------------+-------------------+--------------------+
|  count|           107615|             108128|              108128|
|   mean|36839.03430748502|  6434187.413250962|  1052360.7834973366|
| stddev|974215.3015529716|9.324061726649418E7|3.7760499401841596E7|
|    min|                0|                  0|                   0|
|    max|        100000000|         1430423170|          1428977971|
+-------+-----------------+-------------------+--------------------+



Observez les autres colonnes, posez-vous les bonnes questions : quel cleaning faire pour chaque colonne ? Y a-t-il des colonnes inutiles ? Comment traiter les valeurs manquantes ? A-t-on des données dupliquées ? Quelles sont les valeurs de mes colonnes ? Des répartitions intéressantes ? Des "fuites du futur" (vous entendrez souvent le terme data leakage) ??? Proposez des cleanings à faire sur les données : des groupBy-count, des show, des dropDuplicates, etc.

In [12]:
#dfcasted.groupBy(df.disable_communication).count().orderBy("count", ascending=False).show(100)

dfcasted.select("deadline").dropDuplicates().show()

dfcasted.groupBy(df.country).count().orderBy("count", ascending=False).show(100)
dfcasted.groupBy(df.currency).count().orderBy("count", ascending=False).show(100)
dfcasted.groupBy(df.state_changed_at).count().orderBy("count", ascending=False).show(100)
dfcasted.groupBy("backers_count").count().orderBy("count", ascending=False).show(100)
dfcasted.select("goal", "final_status").show(30)
dfcasted.groupBy("country", "currency").count().orderBy("count",ascending=False).show(50)

+----------+
|  deadline|
+----------+
|1254767400|
|1263597060|
|1274115540|
|1274145060|
|1280372820|
|1281730020|
|1281769200|
|1283292000|
|1286168724|
|1287701689|
|1288926510|
|1289102340|
|1290314164|
|1290704714|
|1291537500|
|1292387613|
|1292400000|
|1293256800|
|1297473364|
|1297726200|
+----------+
only showing top 20 rows

+--------------------+-----+
|             country|count|
+--------------------+-----+
|                  US|91545|
|                  GB| 8743|
|                  CA| 3733|
|                  AU| 1877|
|                  NL|  702|
|               False|  428|
|                  NZ|  354|
|                  SE|  240|
|                  DK|  196|
|                  NO|  113|
|                  IE|  111|
|               999.0|    2|
| rofl-metaphor-56-ep|    1|
|              5000.0|    1|
|up-until-now-reco...|    1|
|sketches-in-time-...|    1|
|in-the-grass-a-sh...|    1|
|dave-afdahl-cliff...|    1|
|kuya-ko-my-big-br...|    1|
|welcome-to-not-mo...|  

AnalysisException: Resolved attribute(s) state_changed_at#26 missing from project_id#17,name#18,desc#19,goal#136,keywords#21,disable_communication#22,country#23,currency#24,deadline#151,state_changed_at#166,created_at#181,launched_at#196,backers_count#211,final_status#226 in operator !Aggregate [state_changed_at#26], [state_changed_at#26, count(1) AS count#750L]. Attribute(s) with the same name appear in the operation: state_changed_at. Please check if the right attribute(s) are used.;
!Aggregate [state_changed_at#26], [state_changed_at#26, count(1) AS count#750L]
+- Project [project_id#17, name#18, desc#19, goal#136, keywords#21, disable_communication#22, country#23, currency#24, deadline#151, state_changed_at#166, created_at#181, launched_at#196, backers_count#211, cast(final_status#30 as int) AS final_status#226]
   +- Project [project_id#17, name#18, desc#19, goal#136, keywords#21, disable_communication#22, country#23, currency#24, deadline#151, state_changed_at#166, created_at#181

Enlevez la colonne disable_communication. Cette colonne est très largement majoritairement à false, il n'y a que 322 true (négligeable), le reste est non-identifié :

In [13]:
df2 = dfcasted.drop("disable_communication")

### Les fuites du futur

Dans les datasets construits a posteriori des évènements, il arrive que des données ne pouvant être connues qu'après la résolution de chaque évènement soient insérées dans le dataset. On a des fuites depuis le futur ! Par exemple, on a ici le nombre de "backers" dans la colonne backers_count. Il s'agit du nombre total de personnes ayant investi dans chaque projet, or ce nombre n'est connu qu'après la fin de la campagne.

Il faut savoir repérer et traiter ces données pour plusieurs raisons :

- Pendant l'entraînement (si on ne les a pas enlevées) elles facilitent le travail du modèle puisqu'elles contiennent des informations directement liées à ce qu'on veut prédire. Par exemple, si backers_count = 0 on est sûr que la campagne a raté.
- Au moment d'appliquer notre modèle, les données du futur ne sont pas présentes (puisqu'elles ne sont pas encore connues). On ne peut donc pas les utiliser comme input pour un modèle.

Ici, pour enlever les données du futur on retire les colonnes backers_count et state_changed_at :

In [14]:
dfNoFuture = df2.drop("backers_count", "state_changed_at")

### Colonnes currency et country

On pourrait penser que les colonnes currency et country sont redondantes, auquel cas on pourrait enlever une des colonnes. Mais c'est oublier par exemple que tous les pays de la zone euro ont la même monnaie ! Il faut donc garder les deux colonnes.

Il semble y avoir des inversions entre ces deux colonnes et du nettoyage à faire. On remarque en particulier que lorsque country = "False" le country à l'air d'être dans currency. On le voit avec la commande

In [15]:
df.filter(df.country == False) \
  .groupBy("currency") \
  .count() \
  .orderBy("count", ascending = False) \
  .show(50)

+--------+-----+
|currency|count|
+--------+-----+
|      US|  405|
|     NOK|  113|
|      GB|   13|
|      CA|    3|
|      AU|    3|
|      NL|    2|
|      NZ|    1|
|      NO|    1|
+--------+-----+



### Créez deux udfs nommées udf_country et udf_currency telles que :
- cleanCountry : si country = "False" prendre la valeur de currency, sinon si country est une chaîne de caractères de taille autre que 2 remplacer par null, et sinon laisser la valeur country actuelle. On veut les résultat dans une nouvelle colonne country2.
- cleanCurrency : si currency.length != 3 currency prend la valeur null, sinon laisser la valeur currency actuelle. On veut les résultats dans une nouvelle colonne currency2.


In [21]:

def cleanCountry(country: str, currency: str) -> str:
  if (country == "False"):
    currency
  else:
    country

def cleanCurrency(currency: str) -> str:
  if (currency != None and len(currency) != 3):
    return None
  else:
    return currency

cleanCountryUdf = udf(cleanCountry)
cleanCurrencyUdf = udf(cleanCurrency)

dfCountry = dfNoFuture.withColumn("country2", cleanCountryUdf("country", "currency")) \
                      .withColumn("currency2", cleanCurrencyUdf("currency")) \
                      .drop("country", "currency")

On a montré ici l'utilisation d'udfs, mais de façon générale toujours privilégier les fonctions déjà codées dans Spark car elles sont optimisées.

Pour une classification, l’équilibrage entre les différentes classes cibles dans les données d’entraînement doit être contrôlé (et éventuellement corrigé). Affichez le nombre d’éléments de chaque classe (colonne final_status). Conservez uniquement les lignes qui nous intéressent pour le modèle, à savoir lorsque final_status vaut 0 (Fail) ou 1 (Success). Les autres valeurs ne sont pas définies et on les enlève. On pourrait toutefois tester en mettant toutes les autres valeurs à 0 en considérant que les campagnes qui ne sont pas un Success sont un Fail.

### Ajouter et manipuler des colonnes

Il est parfois utile d’ajouter des features (colonnes dans un DataFrame) pour aider le modèle lors de son apprentissage. Ici nous allons créer de nouvelles features à partir de celles déjà présentes dans les données. Dans certains cas on peut ajouter des features en allant chercher des sources de données supplémentaires.

Les dates ne sont pas directement exploitables par un modèle sous leur forme initiale dans nos données : il s’agit de timestamps Unix (nombre de secondes depuis le 1er janvier 1970 0h00 UTC). Nous allons traiter ces données pour en extraire des informations pour aider les modèles. Nous allons, entre autres, nous servir des fonctions liées aux dates de l'objet functions.

Ajoutez une colonne days_campaign qui représente la durée de la campagne en jours (le nombre de jours entre launched_at et deadline).

Ajoutez une colonne hours_prepa qui représente le nombre d’heures de préparation de la campagne entre created_at et launched_at. On pourra arrondir le résultat à 3 chiffres après la virgule.

Supprimez les colonnes launched_at, created_at, et deadline, elles ne sont pas exploitables pour un modèle.

Pour exploiter les données sous forme de texte, nous allons commencer par réunir toutes les colonnes textuelles en une seule. En faisant cela, on rend indiscernable le texte du nom de la campagne, de sa description et des keywords, ce qui peut avoir des conséquences sur la qualité du modèle. Mais on cherche à construire ici un premier benchmark de modèle, avec une solution simple qui pourra servir de référence pour des modèles plus évolués.

Mettre les colonnes name, desc, et keywords en minuscules.

Ajoutez une colonne text, qui contient la concaténation des Strings des colonnes name, desc, et keywords. ATTENTION à bien mettre des espaces entre les chaînes de caractères concaténées, car on fera par la suite un split en se servant des espaces entre les mots.

### Valeurs nulles

Il y plusieurs façons de traiter les valeurs nulles pour les rendre exploitables par un modèle. Nous avons déjà vu que parfois les valeurs nulles peuvent être comblées en utilisant les valeurs d’une autre colonne (parce que le dataset a été mal préparé). On peut aussi décider de supprimer les exemples d’entraînement contenant des valeurs nulles, mais on risque de perdre beaucoup de données. On peut également les remplacer par la valeur moyenne ou médiane de la colonne. On peut enfin leur attribuer une valeur particulière, distincte des autres valeurs de la colonne.

Remplacez les valeurs nulles des colonnes days_campaign, hours_prepa, et goal par la valeur -1 et par "unknown" pour les colonnes country2 et currency2.

### Sauvegarder un DataFrame

Sauvegarder le DataFrame final au format parquet sur votre machine :

In [25]:
dfCountry.write.parquet("/home/mgard/Documents/pyspark/cours-spark-telecom/data/out")

22/11/27 21:10:06 WARN MemoryManager: Total allocation exceeds 95,00% (906 992 014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers
22/11/27 21:10:06 WARN MemoryManager: Total allocation exceeds 95,00% (906 992 014 bytes) of heap memory
Scaling row group sizes to 84,47% for 8 writers
22/11/27 21:10:07 WARN MemoryManager: Total allocation exceeds 95,00% (906 992 014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers


Attention ! Lorsqu’on sauvegarde un output en Spark, le résultat est toujours un répertoire contenant un ou plusieurs fichiers. Cela est dû à la nature distribuée de Spark.